##### 코드 수정
- 리뷰에 제공해주신 코드를 바탕으로 코드를 다시 수정했습니다.   
```python
quiz_list = []

# 대화 진행
while True:
    # 1. 퀴즈 생성
    quiz = rag_chain.invoke("퀴즈를 시작하세요.")
# '퀴즈:'로 시작하는 내용만 추출
    quiz_pattern = r"퀴즈: .*"
    all_quizzes = "\n".join(quiz_list)  # 리스트를 문자열로 결합
    quiz_onlys = re.findall(quiz_pattern, all_quizzes)  # 문자열에서 검색
```

- 내용을 추출해오는 곳을 previous_conversation에서 quiz_list로 바꿨습니다.
```python
# '퀴즈:'로 시작하는 내용만 추출
quiz_pattern = r"퀴즈: .*"
all_quizzes = "\n".join(quiz_list)  # 리스트를 문자열로 결합
quiz_onlys = re.findall(quiz_pattern, all_quizzes)  # 문자열에서 검색
```

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pprint import pprint

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

from dotenv import load_dotenv
import os

# .env 파일에서 환경변수 로드
load_dotenv("C:/.env")

llm = ChatOpenAI(model="gpt-4o-mini")

# Selenium 옵션 설정 (헤드리스 모드로 실행)
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
chrome_options.add_argument("--disable-gpu")  # GPU 비활성화 (일부 환경에서 필요)

# WebDriver 경로 설정 (자동 설치)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url_list=[]
txt_list=[]

# 환경변수에 저장된 URL 로드
for i in range(1, 17):  # URL_1 ~ URL_16
    url = os.getenv(f"URL_{i}")
    if url:  # 환경변수가 존재하면 추가
        url_list.append(url)

# 웹페이지 요청
for url in url_list:
    driver.get(url)  # 페이지 로드

    # 특정 요소가 로드될 때까지 기다림 (예: Notion 페이지에서 주요 콘텐츠가 담길 요소)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".notion-page-content"))
        )
    except TimeoutException:
        print(f"페이지 로딩 실패: {url}")
        continue
    
    # 토글이 닫혀 있으면 토글을 열기
    try:
        # 모든 토글 버튼을 찾음 (Ctrl+Alt+T에 해당하는 토글을 찾아서 열기)
        toggle_buttons = driver.find_elements(By.XPATH, "//div[@role='button' and contains(@aria-label, '열기')]")
        
        # 각 토글을 클릭하여 열기
        for button in toggle_buttons:
            button.click()
            time.sleep(1)  # 토글이 열리기 전에 잠깐 대기
        
    except Exception as e:
        print(f"토글을 여는 데 실패했습니다: {e}")

    # 페이지의 HTML 가져오기
    html_code = driver.page_source

    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(html_code, 'html.parser')

    txt = soup.get_text()

    # 1. \xa0를 공백으로 변환
    txt = txt.replace('\xa0', ' ')

    # 2. 정규식을 사용해 \\로 시작하는 LaTeX 명령어 제거
    txt = re.sub(r'\\[a-zA-Z]+\{.*?\}', '', txt)  # \command{...} 형식 제거
    txt = re.sub(r'\\[a-zA-Z]+', '', txt)        # \command 형식 제거

    # 3. 불필요한 공백 제거 (코드 개행 유지를 위해 주석처리)
    # txt = re.sub(r'\s+', ' ', txt).strip()

    # 텍스트만 가져오기
    txt_list.append(txt)


driver.quit()  # 브라우저 종료

In [4]:
# 결과 출력
pprint(txt_list[9])  # 두 번째 URL의 텍스트 내용 출력

('[스파르타코딩클럽] 10강. 지도학습 : 분류모델 - SVM[SCC] 바닥부터 시작하는 머신러닝/[스파르타코딩클럽] 바닥부터 시작하는 '
 '머신러닝 - 3주차/[스파르타코딩클럽] 10강. 지도학습 : 분류모델 - SVM제작:[스파르타코딩클럽] 10강. 지도학습 : 분류모델 - '
 'SVM[수업 목표]SVM(Support Vector Machine)에 대한 개념을 배우고, 데이터를 이용해 실습해 봅니다[목차]01. '
 'SVM 개념02. SVM 실습💡모든 토글을 열고 닫는 단축키\n'
 'Windows : Ctrl + alt + t \n'
 'Mac : ⌘ + ⌥ + t 01. SVM 개념✔️SVM이 무엇인지 알아봅시다1) SVM SVM이란?서포트 벡터 머신(SVM)은 분류와 '
 '회귀 분석에 사용되는 강력한 지도학습 모델데이터를 분류하기 위해 결정 경계(결정 초평면, hyperplane)를 찾아 분류합니다.초평면은 '
 '두 클래스 사이의 최대 마진을 보장하는 방식으로 선택합니다.ALT마진 : 두 클래스 간의 가장 가까운 데이터 포인트 사이의 거리마진 : '
 '두 클래스 간의 가장 가까운 데이터 포인트 사이의 거리\ufeff\n'
 '서포트 벡터 : 결정 초평면에 가장 가까이 위치한 데이터 포인트 - 결정 초평면을 정의합니다서포트 벡터 : 결정 초평면에 가장 가까이 '
 '위치한 데이터 포인트 - 결정 초평면을 정의합니다\ufeff\n'
 '커널 함수 : 데이터를 더 높은 차원으로 매핑하여 선형적으로 분리 할 수 없는 데이터를 분리하게 합니다. 커널 함수 : 데이터를 더 높은 '
 '차원으로 매핑하여 선형적으로 분리 할 수 없는 데이터를 분리하게 합니다. \ufeff\u200b SVM의 목적SVM의 목표는 마진을 '
 '최대화하면서 결정 초평면을 찾아 데이터 포인트를 정확하게 분류하는 것입니다. 이는 일반화 성능을 높이는 데 도움을 '
 '줍니다.w⋅x−b=0   - b = 0 w⋅x−b=0여기서 w는 가중치 벡터, x는 입력 벡터, b는 절편입니

In [5]:
from langchain.schema import Document

# 1. 로드된 문서 전처리(청킹)
docs = ''.join(txt_list)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
str_splits = text_splitter.split_text(docs)

# 2. 문자열 리스트를 Document 객체로 변환
doc_splits = [Document(page_content=str) for str in str_splits]

vectorstore = Chroma.from_documents(documents=doc_splits, embedding=OpenAIEmbeddings())
print(f"청크로 나눠진 후, 청크의 개수: {len(doc_splits)}")

# # 상위 10개의 청크 출력
# print("Top 10 chunks:")
# for i, chunk in enumerate(doc_splits[:10], 1):
#     pprint(f"\nChunk {i}:\n{chunk.page_content}")

retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages([("system", """
    당신은 AI 강사입니다. 아래 context를 기반으로 하나의 퀴즈를 만들어 사용자의 대답을 기다리세요.
    퀴즈는 보기가 있는 객관식 또는 O,X 형태로 출제해주세요. (주로 코드 내용과 관련된 문제를 추천합니다.)
    이후, 사용자의 대답을 확인하고 아래 형식을 바탕으로 피드백을 제공하세요:
    - 정답 여부: "N번" 또는 "예/아니오"
    - 추가 설명: (정답과 관련된 추가 정보를 제공하세요)
    
    Context: {context}
    """)])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
        {"context": retriever | format_docs}
        | prompt
        | llm
        | StrOutputParser()
)

청크로 나눠진 후, 청크의 개수: 421


In [17]:
from datetime import datetime

# 폴더 이름
folder_name = "previous_conversation"

# 폴더가 없으면 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 파일 이름에 타임스탬프 추가
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # "20241126_153045" 형식
file_name = f"conversation_log_{timestamp}.txt"
file_path = os.path.join(folder_name, file_name)

quiz_list = []

# 대화 진행
while True:
    # 1. 퀴즈 생성
    quiz = rag_chain.invoke("퀴즈를 시작하세요.")
    quiz_list.append(quiz)
    print(quiz)
    
    # 2. 사용자 답변 수집
    user_answer = input("답변을 입력하세요: ")
    if user_answer.strip().lower() == "exit":
        print("대화를 종료합니다.")
        break
    print(user_answer)

    # 이전 대화 내용 불러오기
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            previous_conversation = f.read()
    else:
        previous_conversation = ""  # 파일이 없으면 빈 문자열로 시작

    # '퀴즈:'로 시작하는 내용만 추출
    quiz_pattern = r"퀴즈: .*"
    all_quizzes = "\n".join(quiz_list)  # 리스트를 문자열로 결합
    quiz_onlys = re.findall(quiz_pattern, all_quizzes)  # 문자열에서 검색

    # # 필요한 부분 출력
    # print("추출된 퀴즈:")
    # for quiz_only in quiz_onlys:
    #     print(quiz_only)
    
    # 3. 사용자 답변에 대한 피드백 생성
    feedback_prompt = ChatPromptTemplate.from_messages([
        ("system", f"""
    AI 강사로서 다음 퀴즈의 정답 여부를 확인하고 피드백을 제공하세요.
    피드백은 아래와 같은 형식이어야 합니다:
    - 정답 여부: "N번" 또는 "예/아니오"
    - 추가 설명: (정답과 관련된 추가 정보를 제공하세요)
    퀴즈: {{quiz}}
    답변: {{answer}}
    대화 기록: {{quiz_onlys}}
    거절 사유: {{refusal}}
    """)
    ])

    # 피드백 생성 - 키워드 인수로 전달
    feedback_data = feedback_prompt.format(
        quiz=quiz,
        answer=user_answer,
        quiz_onlys=quiz_onlys,
        refusal="None"
    )

    # format 결과를 확인
    print("Formatted Feedback Data:")
    print(feedback_data)

    # 피드백 체인 호출
    feedback = llm.invoke(feedback_data)   # LLM을 직접 호출하여 피드백 생성
    
    print("Feedback:")
    pprint(feedback)
    
    # 대화 내용 저장(파일에 기록)
    with open(file_path, "a", encoding="utf-8") as f:
        f.write(f"Quiz: {quiz}\n")
        f.write(f"User Answer: {user_answer}\n")
        f.write(f"Feedback: {feedback}\n")
        f.write("-" * 50 + "\n")

퀴즈: 나이브베이즈 분류모델에 대한 설명 중 옳은 것을 고르세요.

1. 나이브베이즈 모델은 모든 입력 변수가 서로 독립적이라고 가정한다.
2. 나이브베이즈 모델은 연속형 데이터만 처리할 수 있다.
3. 나이브베이즈 모델은 비지도학습에 속한다.
4. 나이브베이즈 모델은 주로 회귀 문제에 사용된다.

정답을 1, 2, 3, 4 중 하나로 선택해 주세요!
1
Formatted Feedback Data:
System: 
    AI 강사로서 다음 퀴즈의 정답 여부를 확인하고 피드백을 제공하세요.
    피드백은 아래와 같은 형식이어야 합니다:
    - 정답 여부: "N번" 또는 "예/아니오"
    - 추가 설명: (정답과 관련된 추가 정보를 제공하세요)
    퀴즈: 퀴즈: 나이브베이즈 분류모델에 대한 설명 중 옳은 것을 고르세요.

1. 나이브베이즈 모델은 모든 입력 변수가 서로 독립적이라고 가정한다.
2. 나이브베이즈 모델은 연속형 데이터만 처리할 수 있다.
3. 나이브베이즈 모델은 비지도학습에 속한다.
4. 나이브베이즈 모델은 주로 회귀 문제에 사용된다.

정답을 1, 2, 3, 4 중 하나로 선택해 주세요!
    답변: 1
    대화 기록: ['퀴즈: 나이브베이즈 분류모델에 대한 설명 중 옳은 것을 고르세요.']
    거절 사유: None
    
Feedback:
AIMessage(content='- 정답 여부: "1"\n- 추가 설명: 나이브베이즈 모델은 베이즈 정리를 기반으로 하며, 모든 입력 변수가 서로 독립적이라는 가정을 합니다. 이 가정은 "나이브(naive)"라는 이름의 유래이기도 합니다. 나이브베이즈는 주로 분류 문제에 사용되며, 이론적으로는 연속형 데이터와 범주형 데이터 모두 처리할 수 있지만, 일반적으로는 범주형 데이터에 더 적합합니다. 비지도학습이 아닌 지도학습에 속하며, 회귀 문제에는 보통 사용되지 않습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_us